## Installs

In [1]:
#Install packages to apply SVD and Bayesian Factorization Machine
!pip install surprise

!pip install myfm

## Imports

In [8]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To apply SVD
import surprise

# To apply Factorization Machines
import myfm
from myfm import RelationBlock
from myfm.utils.encoders import(DataFrameEncoder,MultipleValuesToSparseEncoder,CategoryValueToSparseEncoder)

# To do train-test split for evaluation
from sklearn.model_selection import train_test_split

## Load Data

In [3]:
#Function to Extract users, movies and ratings from raw data
def extract_users_items_predictions(data_pd):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    predictions = data_pd.Prediction.values
    return users, movies, predictions

#Load raw data
data_pd = pd.read_csv('data/data_train.csv')

#Do Train-Test Split
train, val = train_test_split(data_pd, test_size=0.1,random_state=42)

#Extract users, movies and ratings from raw data
total_users, total_movies, total_pred = extract_users_items_predictions(data_pd)
train_users, train_movies, train_pred = extract_users_items_predictions(train)
val_users, val_movies, val_pred = extract_users_items_predictions(val)

#Store total data
ratings_dict_total = {'userID': total_users,'movieID': total_movies,'rating': total_pred}
df_total = pd.DataFrame(ratings_dict_total)

#Store train data
ratings_dict_train = {'userID': train_users,'movieID': train_movies,'rating': train_pred}
df_train = pd.DataFrame(ratings_dict_train)

#Store validation data
ratings_dict_test = {'userID': val_users,'movieID': val_movies,'rating': val_pred}
df_test = pd.DataFrame(ratings_dict_test)

## Apply SVD 

In [5]:
def surprise_svd(df,number_of_users,number_of_movies):
    
    # Transform Data in Desired Format
    reader = surprise.Reader(rating_scale=(0,5.5))
    surprise_data = surprise.Dataset.load_from_df(df[['userID', 'movieID', 'rating']], reader=reader)
    trainset = surprise_data.build_full_trainset()
    
    #Apply SVD
    algo = surprise.SVD(n_factors=3, n_epochs=50, random_state=1234, biased=False)
    algo.fit(trainset)
    
    #Get User/Movie Factors
    U = algo.pu
    Q = algo.qi
    return U,Q

U,Q = surprise_svd(df_train,10000,1000)

## Get cosine and implicit user/movie Features

In [6]:
'''Code adapted from https://github.com/tohtsky/myFM/blob/main/examples/ml-100k-extended.ipynb'''

#Perform Cosine Similarity
def cosine(a,b):
    return np.dot(a,b) /(np.linalg.norm(a) * np.linalg.norm(b))

#Utility function to convert list to str
def string_agg(int_list) -> str:
    return ','.join([str(y) for y in int_list])

#Generate User Affinity
user_dict = {}
for i in range(10000):
    user_cosine = []
    for j in range(1000):
        user_cosine.append(cosine(U[i],Q[j])) 
    user_cosine.sort(reverse=True)
    user_dict[i] = string_agg(user_cosine[:100])

#Generate Movie Affinity
movie_dict = {}
for i in range(1000):
    movie_cosine = []
    for j in range(10000):
        movie_cosine.append(cosine(Q[i],U[j]))
    movie_cosine.sort(reverse=True)
    movie_dict[i] = string_agg(movie_cosine[:100])
    
#Store the Affinitites
user_f = pd.DataFrame.from_dict(user_dict.items())
user_f.columns = ['userID', 'user_cosine_f']

movie_f = pd.DataFrame.from_dict(movie_dict.items())
movie_f.columns = ['movieID', 'movie_cosine_f']

#Get & Store Implicit Features
user_f['user_implicit_features'] = df_train.groupby('userID').movieID.agg(string_agg).reindex(user_f.index).fillna('')
movie_f['movie_implicit_features'] = df_train.groupby('movieID').userID.agg(string_agg).reindex(movie_f.index).fillna('')

# Transform Data in myfm Format

In [10]:
#Create User Encoder
user_encoder = DataFrameEncoder().add_column(
    'userID', CategoryValueToSparseEncoder(user_f.userID)
    )

user_encoder.add_column(
        'user_implicit_features',
        MultipleValuesToSparseEncoder(user_f.user_implicit_features, normalize=True)
    )

user_encoder.add_column(
        'user_cosine_f',
        MultipleValuesToSparseEncoder(user_f.user_cosine_f, normalize=True)
    )

#Create Movie Encoder
movie_encoder = DataFrameEncoder().add_column(
    'movieID', CategoryValueToSparseEncoder(movie_f.movieID)
    )
movie_encoder.add_column(
        'movie_implicit_features',
        MultipleValuesToSparseEncoder(movie_f.movie_implicit_features, normalize=True)
    )
movie_encoder.add_column(
        'movie_cosine_f',
        MultipleValuesToSparseEncoder(movie_f.movie_cosine_f, normalize=True)
    )


def augment_user_id(user_ids):
    return user_encoder.encode_df(
        user_f.reindex(user_ids).reset_index()
    )

def augment_movie_id(movie_ids):
    return movie_encoder.encode_df(
        movie_f.reindex(movie_ids).reset_index()
    )


#Create Train/Test Blocks
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks),(df_test, test_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

## Evaluation and Parameter Tuning

In [ ]:
# Return root mean square error metric
def RMSE(x: np.ndarray, y: np.ndarray) -> float:
    return np.sqrt(np.average((x - y) ** 2))

# Function to do parameter tuning
def validation(user_encoder,movie_encoder,df_train,train_blocks,test_blocks,val_predictions):
    group_shapes = user_encoder.encoder_shapes + movie_encoder.encoder_shapes

    rank = [8,10,12,14]
    n_iter = [100,200,300,400,500]
    n_samples = [95,195,295,395,495]

    best_rmse = 1
    best_r = 0
    best_n = 0
    best_s = 0

    for r in rank:
        for i in range(len(n_iter)):
            fm = myfm.MyFMRegressor(rank=r,random_seed=1234)
            fm.fit(None, df_train.rating.values, X_rel=train_blocks, 
                   group_shapes=group_shapes,n_iter=n_iter[i], n_kept_samples=n_samples[i])
            test_predictions = fm.predict(None, test_blocks)
            rmse = RMSE(val_predictions,test_predictions)
            if rmse < best_rmse:
                best_rmse = rmse
                best_r = r
                best_n = n_iter[i]
                best_s = n_samples[i]
                print(best_rmse)
                
    return best_rmse,best_r,best_n,best_s

best_rmse,best_r,best_n,best_s = validation(user_encoder,movie_encoder,df_train,train_blocks,test_blocks,val_pred)

alpha = 1.11 w0 = 3.62 : 100%|████████████████| 100/100 [01:35<00:00,  1.05it/s]


0.9730970487787145


alpha = 1.12 w0 = 3.59 : 100%|████████████████| 200/200 [03:11<00:00,  1.04it/s]


0.9727125011087001


alpha = 1.11 w0 = 3.57 : 100%|████████████████| 300/300 [04:53<00:00,  1.02it/s]


...

alpha = 1.11 w0 = 3.57 : 100%|████████████████| 400/400 [06:27<00:00,  1.03it/s]


0.9723171121680038


alpha = 1.11 w0 = 3.59 : 100%|████████████████| 500/500 [08:46<00:00,  1.05s/it]


0.9722562612737108


alpha = 1.13 w0 = 3.67 : 100%|████████████████| 100/100 [02:48<00:00,  1.42s/it]

# Generate Features & Train the model on Total Data

In [ ]:
U,Q = surprise_svd(df_total,10000,1000)

#Generate User Affinity
user_dict = {}
for i in range(10000):
    user_cosine = []
    for j in range(1000):
        user_cosine.append(cosine(U[i],Q[j])) 
    user_cosine.sort(reverse=True)
    user_dict[i] = string_agg(user_cosine[:100])

#Generate Movie Affinity
movie_dict = {}
for i in range(1000):
    movie_cosine = []
    for j in range(10000):
        movie_cosine.append(cosine(Q[i],U[j]))
    movie_cosine.sort(reverse=True)
    movie_dict[i] = string_agg(movie_cosine[:100])
    
#Store the Affinitites
user_f = pd.DataFrame.from_dict(user_dict.items())
user_f.columns = ['userID', 'user_cosine_f']

movie_f = pd.DataFrame.from_dict(movie_dict.items())
movie_f.columns = ['movieID', 'movie_cosine_f']


#Get & Store Implicit Features on Total Data
user_f['user_implicit_features'] = df_total.groupby('userID').movieID.agg(string_agg).reindex(user_f.index).fillna('')
movie_f['movie_implicit_features'] = df_total.groupby('movieID').userID.agg(string_agg).reindex(movie_f.index).fillna('')

#Create User Encoder for total data
user_encoder = DataFrameEncoder().add_column(
    'userID', CategoryValueToSparseEncoder(user_f.userID)
    )

user_encoder.add_column(
        'user_implicit_features',
        MultipleValuesToSparseEncoder(user_f.user_implicit_features, normalize=True)
    )

user_encoder.add_column(
        'user_cosine_f',
        MultipleValuesToSparseEncoder(user_f.user_cosine_f, normalize=True)
    )

#Create Movie Encoder for total data
movie_encoder = DataFrameEncoder().add_column(
    'movieID', CategoryValueToSparseEncoder(movie_f.movieID)
    )
movie_encoder.add_column(
        'movie_implicit_features',
        MultipleValuesToSparseEncoder(movie_f.movie_implicit_features, normalize=True)
    )
movie_encoder.add_column(
        'movie_cosine_f',
        MultipleValuesToSparseEncoder(movie_f.movie_cosine_f, normalize=True)
    )


#Generate Train Blocks on total data
group_shapes = user_encoder.encoder_shapes + movie_encoder.encoder_shapes

total_train_blocks = []
for source, target in [(df_total, total_train_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )


#Train the model
fm = myfm.MyFMRegressor(rank=best_r,random_seed=1234)
fm.fit(None, df_total.rating.values, X_rel=train_blocks,
    group_shapes=group_shapes,
    n_iter=best_n, n_kept_samples=best_s
)

## Generate predictions for Submission

In [ ]:
#Load Submission File
sub_pd = pd.read_csv('data/sampleSubmission.csv',index_col='Id')
sub_users, sub_movies, sub_pred = extract_users_items_predictions(sub_pd)
sub_test_ratings_dict = {'userID': sub_users,'movieID': sub_movies,'rating': subpred}
sub_df = pd.DataFrame(sub_test_ratings_dict)

#Generate Submission Test Blocks
sub_test_blocks = []
for source, target in [(sub_df, sub_test_blocks)]:
    unique_users, user_map = np.unique(source.userID, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movieID, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

#Generate Predictions and create submission csv    
predictions = fm.predict(None, sub_test_blocks)
sub_pd['Prediction'] = test_predictions
sub_pd.to_csv("submission_bayesian_svdpp_flipped_cosine.csv")